In [60]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

In [61]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [62]:
epochs = 50
batch_size = 4
learning_rate = 0.001

In [63]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

In [64]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)

val_split = 0.2

train_indices, val_indices = train_test_split(list(range(len(train_data))), test_size=val_split, random_state=42)

train_subset = Subset(train_data, train_indices)
val_subset = Subset(train_data, val_indices)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [65]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [66]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [67]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
steps = len(train_loader)

patience = 3
min_delta = 0.001
best_loss = np.Inf
counter = 0

In [68]:
for epoch in range(epochs):
    model.train() 
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        #if (i+1) % 2000 == 0:
        #    print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{steps}], Loss: {loss.item():.4f}')

    model.eval()  
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    running_loss /= len(train_loader)
    val_loss /= len(val_loader)

    if val_loss < best_loss - min_delta:
        best_loss = val_loss
        counter = 0  
    else:
        counter += 1

    print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss:.4f}, Validation Loss: {val_loss:.4f}')

    if counter >= patience:
        print(f'Early stopping after {epoch+1} epochs.')
        break

Epoch [1/50], Training Loss: 2.2584, Validation Loss: 2.0464
Epoch [2/50], Training Loss: 1.8763, Validation Loss: 1.7203
Epoch [3/50], Training Loss: 1.6452, Validation Loss: 1.5644
Epoch [4/50], Training Loss: 1.5322, Validation Loss: 1.4910
Epoch [5/50], Training Loss: 1.4505, Validation Loss: 1.3862
Epoch [6/50], Training Loss: 1.3758, Validation Loss: 1.3369
Epoch [7/50], Training Loss: 1.3094, Validation Loss: 1.2494
Epoch [8/50], Training Loss: 1.2562, Validation Loss: 1.2022
Epoch [9/50], Training Loss: 1.2092, Validation Loss: 1.1510
Epoch [10/50], Training Loss: 1.1701, Validation Loss: 1.1201
Epoch [11/50], Training Loss: 1.1335, Validation Loss: 1.1136
Epoch [12/50], Training Loss: 1.1004, Validation Loss: 1.0486
Epoch [13/50], Training Loss: 1.0707, Validation Loss: 1.0315
Epoch [14/50], Training Loss: 1.0424, Validation Loss: 1.0012
Epoch [15/50], Training Loss: 1.0152, Validation Loss: 0.9648
Epoch [16/50], Training Loss: 0.9886, Validation Loss: 0.9419
Epoch [17/50], Tr

In [69]:
with torch.no_grad():
    num_correct = 0
    num_samples = 0
    num_correct_class = [0 for i in range(10)]
    num_class_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        num_samples += labels.size(0)
        num_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]

            if (label == pred):
                num_correct_class[label] += 1
            
            num_class_samples[label] += 1

In [70]:
accuracy = 100 * num_correct / num_samples
print(f'Accuracy: {accuracy} %')

Accuracy: 64.48 %


In [71]:
for i in range(10):
    accuracy = 100 * num_correct_class[i] / num_class_samples[i]
    print(f'Accuracy of {classes[i]}: {accuracy} %')

Accuracy of plane: 70.2 %
Accuracy of car: 77.9 %
Accuracy of bird: 52.6 %
Accuracy of cat: 42.3 %
Accuracy of deer: 57.7 %
Accuracy of dog: 50.5 %
Accuracy of frog: 72.4 %
Accuracy of horse: 70.2 %
Accuracy of ship: 78.7 %
Accuracy of truck: 72.3 %
